# Venus Altimetry Modelling - Process Data
The Open University<br>
Module SXS841 Masters level project in physical sciences<br>
***
End of Module Assignment (EMA) 01<br>
Stuart Gavin Smith<br>
H1872126<br>
03 January 2025<br>
Version:1.0<br>
***

In [ ]:
from urllib.request import urlretrieve
import calendar
import datetime
import sys
import warnings
warnings.simplefilter('ignore', FutureWarning)
from pandas import *
from numpy import  *
import xml.etree.ElementTree as ET
import struct
%matplotlib inline

#URL for Magellan data
dataurl = r'https://pds-geosciences.wustl.edu/mgn/urn-nasa-pds-magellan_arcdr/data/'

#Set local folder
tempfilesdirectory = 'C:/Users/stuar/OneDrive/Open University/SXS841/Venus Altimetry Modelling/Data/'

#Download the CSV index file for the Radar Altimetry Collection
COLLECTION_DATA ='https://pds-geosciences.wustl.edu/mgn/urn-nasa-pds-magellan_arcdr/data/collection_data_altimetry_inventory.csv'

#Read CSV data into dataframe, on error, print warning and stop kernel.
try:
    #Filename is not stored in the CSV, but a column of character 'P', will use this to store the filename
    dfCollection=read_csv(COLLECTION_DATA, header=None,names=['FILENAME', 'DATASTRING'])
except:
    print("COLLECTION_DATA Venus data does not exist, or some other error")
    Error = True
    exit()
    
#Parse each entry in index file, an example being "urn:nasa:pds:magellan_arcdr:data_altimetry:adf00376_3::1.0"    
def parse_collection(datastring):
    datastring = datastring[1:].split(':')[5]
    return datastring

#Parse the filename in the inventory to work out the directory the data sits
def add_directory(datastring):
    filename = datastring
    directorystring = datastring.replace("adf", "")
    directorynumber = int(directorystring[0:].split('_')[0])
    #divide by 20 to work out the directory range (integer division)
    if (directorynumber % 20) > 0:
        dir_upper = directorynumber // 20
        dir_upper = (dir_upper + 1) * 20
        dir_lower = (dir_upper - 20) + 1
    else:
        dir_upper = directorynumber // 20
        dir_upper = (dir_upper) * 20
        dir_lower = (dir_upper - 20) + 1
    dir_upper = "{:0>{}}".format(dir_upper, 4)
    dir_lower = "{:0>{}}".format(dir_lower, 4)
    tempdirectory = str(dir_lower) + '_' + str(dir_upper)
    datastring = dataurl + tempdirectory + '/' + filename
    return datastring

#Parse the data frame string entry
dfCollection['DATASTRING'] = dfCollection['DATASTRING'].apply(parse_collection)
dfCollection['FILENAME'] = dfCollection['DATASTRING']

#Add the directory to filename
dfCollection['DATASTRING'] = dfCollection['DATASTRING'].apply(add_directory) 
                            
#Create list storage for radar data (faster than dataframe)
VLat = []
VLong = []
VAlt = []

#Cycle through the list of data files, perform some confidence checks on the XML, add data to the dataframe 
tempcount = 0

for index, row in dfCollection.iterrows():
    
    #Set the source and target filenames
    xmlfilename = tempfilesdirectory + row['FILENAME'] + ".xml"
    urlxmlfilename = row['DATASTRING'] + '.xml'

    #Get the file from the server
    urlretrieve(urlxmlfilename, xmlfilename)

    # Parse the XML file
    tree = ET.parse(xmlfilename)
    root = tree.getroot()

    #Get the XML namespace
    namespaces = {'ns': root.tag.split('}')[0].strip('{')}

    #Get the data filename
    for general_data in root.iter('{http://pds.nasa.gov/pds4/pds/v1}File'):
        data_filename = general_data.find('{http://pds.nasa.gov/pds4/pds/v1}file_name').text

    for general_data in root.iter('{http://pds.nasa.gov/pds4/pds/v1}Record_Binary'):
        record_length = int(general_data.find('{http://pds.nasa.gov/pds4/pds/v1}record_length').text)

    for general_data in root.iter('{http://pds.nasa.gov/pds4/pds/v1}Table_Binary'):
        name = general_data.find('{http://pds.nasa.gov/pds4/pds/v1}name').text
        records = int(general_data.find('{http://pds.nasa.gov/pds4/pds/v1}records').text)

    for field_binary in root.iter('{http://pds.nasa.gov/pds4/pds/v1}Field_Binary'):
        name = field_binary.find('{http://pds.nasa.gov/pds4/pds/v1}name').text
        field_location = field_binary.find('{http://pds.nasa.gov/pds4/pds/v1}field_location').text
        if str(name) == 'Footprint_Longitude':
            Longitude_Offset = int(field_location)
        if str(name) == 'Footprint_Latitude':
            Latitude_Offset = int(field_location)
        if str(name) == 'Derived_Planetary_Radius':
            Derived_Planetary_Radius_Offset = int(field_location)        

    #Get the data file from the server
    localdatafilename = tempfilesdirectory + data_filename
    urldatafilename = row['DATASTRING'] + '.dat'

    #Check the calculated data file matches the XML data file
    if row['FILENAME'] + ".dat" != data_filename:
        print ('Error - mis-matched names ' + row['FILENAME'] + ".dat" + " == " + data_filename)
    urlretrieve(urldatafilename, localdatafilename)
    
    #Process each record in the file
    with open(localdatafilename, 'rb') as file:
        count = 0
        while (count < records):
            file.seek((Longitude_Offset+(count*record_length))-1)
            bytes_data = file.read(4)
            longitude_float_value = struct.unpack('f', bytes_data)[0]        
            VLong.append(longitude_float_value)
            file.seek((Latitude_Offset+(count*record_length))-1)
            bytes_data = file.read(4)
            latitude_float_value = struct.unpack('f', bytes_data)[0]          
            VLat.append(latitude_float_value)
            file.seek((Derived_Planetary_Radius_Offset+(count*record_length))-1)
            bytes_data = file.read(4)
            dpradius_float_value = struct.unpack('f', bytes_data)[0]  
            VAlt.append(dpradius_float_value)
            if count == 0:
                print('Processing: ' + data_filename)
            count = count + 1

#convert the data to a data frame
dfAltData = DataFrame()
dfAltData['Long']= VLong
dfAltData['Lat'] = VLat
dfAltData['Alt'] = VAlt

#Save the processed altimetry data
AltimetryDataFilename = tempfilesdirectory + "magellan_altimetry.csv"
dfAltData.to_csv('AltimetryDataFilename.csv', index=False)


Processing: adf00376_3.dat
Processing: adf00377_3.dat
Processing: adf00378_3.dat
Processing: adf00379_3.dat
Processing: adf00380_3.dat
Processing: adf00381_3.dat
Processing: adf00382_3.dat
Processing: adf00383_3.dat
Processing: adf00384_3.dat
Processing: adf00385_3.dat
Processing: adf00386_3.dat
Processing: adf00387_3.dat
Processing: adf00388_3.dat
Processing: adf00389_3.dat
Processing: adf00390_3.dat
Processing: adf00391_3.dat
Processing: adf00392_3.dat
Processing: adf00393_3.dat
Processing: adf00394_3.dat
Processing: adf00395_3.dat
Processing: adf00396_3.dat
Processing: adf00397_3.dat
Processing: adf00398_3.dat
Processing: adf00399_3.dat
Processing: adf00400_3.dat
Processing: adf00401_3.dat
Processing: adf00402_3.dat
Processing: adf00403_3.dat
Processing: adf00404_3.dat
Processing: adf00405_3.dat
Processing: adf00406_3.dat
Processing: adf00407_3.dat
Processing: adf00408_3.dat
Processing: adf00409_3.dat
Processing: adf00410_3.dat
Processing: adf00411_3.dat
Processing: adf00412_3.dat
P

Processing: adf00622_4.dat
Processing: adf00623_3.dat
Processing: adf00623_4.dat
Processing: adf00624_3.dat
Processing: adf00624_4.dat
Processing: adf00625_3.dat
Processing: adf00625_4.dat
Processing: adf00626_3.dat
Processing: adf00626_4.dat
Processing: adf00627_3.dat
Processing: adf00627_4.dat
Processing: adf00628_3.dat
Processing: adf00628_4.dat
Processing: adf00629_3.dat
Processing: adf00629_4.dat
Processing: adf00630_3.dat
Processing: adf00630_4.dat
Processing: adf00631_3.dat
Processing: adf00631_4.dat
Processing: adf00632_3.dat
Processing: adf00632_4.dat
Processing: adf00633_3.dat
Processing: adf00633_4.dat
Processing: adf00634_3.dat
Processing: adf00634_4.dat
Processing: adf00635_3.dat
Processing: adf00635_4.dat
Processing: adf00636_3.dat
Processing: adf00636_4.dat
Processing: adf00637_3.dat
Processing: adf00637_4.dat
Processing: adf00638_3.dat
Processing: adf00638_4.dat
Processing: adf00639_3.dat
Processing: adf00639_4.dat
Processing: adf00640_3.dat
Processing: adf00640_4.dat
P

Processing: adf00893_4.dat
Processing: adf00894_3.dat
Processing: adf00894_4.dat
Processing: adf00895_3.dat
Processing: adf00895_4.dat
Processing: adf00896_3.dat
Processing: adf00896_4.dat
Processing: adf00897_3.dat
Processing: adf00897_4.dat
Processing: adf00898_3.dat
Processing: adf00898_4.dat
Processing: adf00899_3.dat
Processing: adf00899_4.dat
Processing: adf00900_3.dat
Processing: adf00900_4.dat
Processing: adf00901_3.dat
Processing: adf00901_4.dat
Processing: adf00902_3.dat
Processing: adf00902_4.dat
Processing: adf00903_3.dat
Processing: adf00903_4.dat
Processing: adf00904_3.dat
Processing: adf00904_4.dat
Processing: adf00905_3.dat
Processing: adf00905_4.dat
Processing: adf00906_3.dat
Processing: adf00906_4.dat
Processing: adf00907_3.dat
Processing: adf00907_4.dat
Processing: adf00908_3.dat
Processing: adf00908_4.dat
Processing: adf00909_3.dat
Processing: adf00909_4.dat
Processing: adf00910_3.dat
Processing: adf00910_4.dat
Processing: adf00911_3.dat
Processing: adf00911_4.dat
P

Processing: adf01049_3.dat
Processing: adf01049_4.dat
Processing: adf01050_3.dat
Processing: adf01050_4.dat
Processing: adf01051_3.dat
Processing: adf01051_4.dat
Processing: adf01052_3.dat
Processing: adf01052_4.dat
Processing: adf01053_3.dat
Processing: adf01053_4.dat
Processing: adf01054_3.dat
Processing: adf01054_4.dat
Processing: adf01055_3.dat
Processing: adf01055_4.dat
Processing: adf01056_3.dat
Processing: adf01056_4.dat
Processing: adf01057_3.dat
Processing: adf01057_4.dat
Processing: adf01058_3.dat
Processing: adf01058_4.dat
Processing: adf01059_3.dat
Processing: adf01059_4.dat
Processing: adf01060_3.dat
Processing: adf01061_3.dat
Processing: adf01062_3.dat
Processing: adf01063_3.dat
Processing: adf01064_3.dat
Processing: adf01065_3.dat
Processing: adf01066_3.dat
Processing: adf01067_3.dat
Processing: adf01068_3.dat
Processing: adf01069_3.dat
Processing: adf01070_3.dat
Processing: adf01071_3.dat
Processing: adf01072_3.dat
Processing: adf01073_3.dat
Processing: adf01074_3.dat
P

Processing: adf01342_2.dat
Processing: adf01343_2.dat
Processing: adf01344_2.dat
Processing: adf01345_2.dat
Processing: adf01346_2.dat
Processing: adf01347_2.dat
Processing: adf01348_2.dat
Processing: adf01349_2.dat
Processing: adf01350_2.dat
Processing: adf01351_2.dat
Processing: adf01352_2.dat
Processing: adf01353_2.dat
Processing: adf01354_2.dat
Processing: adf01355_2.dat
Processing: adf01356_2.dat
Processing: adf01357_2.dat
Processing: adf01358_2.dat
Processing: adf01359_2.dat
Processing: adf01360_2.dat
Processing: adf01361_2.dat
Processing: adf01362_2.dat
Processing: adf01363_2.dat
Processing: adf01364_2.dat
Processing: adf01365_2.dat
Processing: adf01366_2.dat
Processing: adf01367_2.dat
Processing: adf01368_2.dat
Processing: adf01369_2.dat
Processing: adf01370_2.dat
Processing: adf01371_2.dat
Processing: adf01372_2.dat
Processing: adf01373_2.dat
Processing: adf01374_2.dat
Processing: adf01375_2.dat
Processing: adf01376_2.dat
Processing: adf01377_2.dat
Processing: adf01378_2.dat
P

...

Processing: adf03233_1.dat
Processing: adf03234_1.dat
Processing: adf03235_1.dat
Processing: adf03236_1.dat
Processing: adf03237_1.dat
Processing: adf03238_1.dat
Processing: adf03239_1.dat
Processing: adf03240_1.dat
Processing: adf03241_1.dat
Processing: adf03242_1.dat
Processing: adf03243_1.dat
Processing: adf03244_1.dat
Processing: adf03245_1.dat
Processing: adf03246_1.dat
Processing: adf03247_1.dat
Processing: adf03248_1.dat
Processing: adf03249_1.dat
Processing: adf03250_1.dat
Processing: adf03251_1.dat
Processing: adf03252_1.dat
Processing: adf03253_1.dat
Processing: adf03254_1.dat
Processing: adf03255_1.dat
Processing: adf03257_1.dat
Processing: adf03258_1.dat
Processing: adf03259_1.dat
Processing: adf03260_1.dat
Processing: adf03261_1.dat
Processing: adf03262_1.dat
Processing: adf03263_1.dat
Processing: adf03264_1.dat
Processing: adf03265_1.dat
Processing: adf03266_1.dat
Processing: adf03267_1.dat
Processing: adf03268_1.dat
Processing: adf03269_1.dat
Processing: adf03270_1.dat
P

Processing: adf03538_1.dat
Processing: adf03539_1.dat
Processing: adf03540_1.dat
Processing: adf03541_1.dat
Processing: adf03542_1.dat
Processing: adf03543_1.dat
Processing: adf03544_1.dat
Processing: adf03545_1.dat
Processing: adf03546_1.dat
Processing: adf03547_1.dat
Processing: adf03548_1.dat
Processing: adf03549_1.dat
Processing: adf03550_1.dat
Processing: adf03551_1.dat
Processing: adf03552_1.dat
Processing: adf03553_1.dat
Processing: adf03554_1.dat
Processing: adf03555_1.dat
Processing: adf03556_1.dat
Processing: adf03557_1.dat
Processing: adf03558_1.dat
Processing: adf03559_1.dat
Processing: adf03560_1.dat
Processing: adf03561_1.dat
Processing: adf03562_1.dat
Processing: adf03563_1.dat
Processing: adf03564_1.dat
Processing: adf03565_1.dat
Processing: adf03566_1.dat
Processing: adf03567_1.dat
Processing: adf03568_1.dat
Processing: adf03569_1.dat
Processing: adf03570_1.dat
Processing: adf03571_1.dat
Processing: adf03572_1.dat
Processing: adf03573_1.dat
Processing: adf03574_1.dat
P

Processing: adf03852_1.dat
Processing: adf03853_1.dat
Processing: adf03854_1.dat
Processing: adf03855_1.dat
Processing: adf03859_1.dat
Processing: adf03860_1.dat
Processing: adf03861_1.dat
Processing: adf03862_1.dat
Processing: adf03863_1.dat
Processing: adf03864_1.dat
Processing: adf03865_1.dat
Processing: adf03866_1.dat
Processing: adf03867_1.dat
Processing: adf03868_1.dat
Processing: adf03869_1.dat
Processing: adf03870_1.dat
Processing: adf03871_1.dat
Processing: adf03872_1.dat
Processing: adf03873_1.dat
Processing: adf03874_1.dat
Processing: adf03875_1.dat
Processing: adf03876_1.dat
Processing: adf03877_1.dat
Processing: adf03878_1.dat
Processing: adf03879_1.dat
Processing: adf03880_1.dat
Processing: adf03883_1.dat
Processing: adf03889_1.dat
Processing: adf03954_1.dat
Processing: adf03955_1.dat
Processing: adf03956_1.dat
Processing: adf03957_1.dat
Processing: adf03958_1.dat
Processing: adf04031_1.dat
Processing: adf04032_1.dat
Processing: adf04033_1.dat
Processing: adf04034_1.dat
P

Processing: adf04309_1.dat
Processing: adf04310_1.dat
Processing: adf04311_1.dat
Processing: adf04319_1.dat
Processing: adf04320_1.dat
Processing: adf04321_1.dat
Processing: adf04322_1.dat
Processing: adf04323_1.dat
Processing: adf04324_1.dat
Processing: adf04325_1.dat
Processing: adf04326_1.dat
Processing: adf04327_1.dat
Processing: adf04328_1.dat
Processing: adf04329_1.dat
Processing: adf04330_1.dat
Processing: adf04331_1.dat
Processing: adf04332_1.dat
Processing: adf04334_1.dat
Processing: adf04335_1.dat
Processing: adf04336_1.dat
Processing: adf04337_1.dat
Processing: adf04338_1.dat
Processing: adf04348_1.dat
Processing: adf04350_1.dat
Processing: adf04351_1.dat
Processing: adf04352_1.dat
Processing: adf04353_1.dat
Processing: adf04355_1.dat
Processing: adf04357_1.dat
Processing: adf04358_1.dat
Processing: adf04360_1.dat
Processing: adf04361_1.dat
Processing: adf04365_1.dat
Processing: adf04366_1.dat
Processing: adf04367_1.dat
Processing: adf04368_1.dat
Processing: adf04369_1.dat
P

Processing: adf05248_1.dat
Processing: adf05249_1.dat
Processing: adf05635_1.dat
Processing: adf05636_1.dat
Processing: adf05639_1.dat
Processing: adf05640_1.dat
Processing: adf05641_1.dat
Processing: adf05642_1.dat
Processing: adf05643_1.dat
Processing: adf05644_1.dat
Processing: adf05646_1.dat
Processing: adf05647_1.dat
Processing: adf05648_1.dat
Processing: adf05649_1.dat
Processing: adf05650_1.dat
Processing: adf05654_1.dat
Processing: adf05661_1.dat
Processing: adf05662_1.dat
Processing: adf05663_1.dat
Processing: adf05665_1.dat
Processing: adf05666_1.dat
Processing: adf05667_1.dat
Processing: adf05670_1.dat
Processing: adf05671_1.dat
Processing: adf05672_1.dat
Processing: adf05673_1.dat
Processing: adf05674_1.dat
Processing: adf05675_1.dat
Processing: adf05676_1.dat
Processing: adf05677_1.dat
Processing: adf05678_1.dat
Processing: adf05679_1.dat
Processing: adf05680_1.dat
Processing: adf05681_1.dat
Processing: adf05682_1.dat
Processing: adf05683_1.dat
Processing: adf05684_1.dat
P